In [6]:
# module import

import warnings
import gym_trading_env  # noqa
import gymnasium as gym
import numpy as np
import pandas as pd
import torch
from gym_trading_env.renderer import Renderer
from stable_baselines3 import DQN
from sb3_contrib import QRDQN
from sb3_contrib.qrdqn import MlpPolicy
from stable_baselines3.common.utils import get_device
# from stable_baselines3.dqn import MlpPolicy
from tqdm import TqdmExperimentalWarning

In [ ]:
print(torch.cuda.is_available())
get_device()

In [7]:
# Preprocess


def SMA(df, ndays):
    SMA = pd.Series(df.close.rolling(ndays).mean(), name="SMA_" + str(ndays))
    return SMA.astype(float).round(2)


def BBANDS(df, n):
    MA = df.close.rolling(window=n).mean()
    SD = df.close.rolling(window=n).std()
    upperBand = MA + (2 * SD)
    lowerBand = MA - (2 * SD)
    return upperBand.astype(float).round(2), lowerBand.astype(float).round(2)


def RSI(df, periods=14):
    close_delta = df.close.diff()
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
    ma_down = down.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()

    _rsi = ma_up / ma_down
    return (100 - (100 / (1 + _rsi))).astype(float).round(2)


def MACD(df):
    k = df["close"].ewm(span=12, adjust=False, min_periods=12).mean()
    d = df["close"].ewm(span=26, adjust=False, min_periods=26).mean()
    macd = k - d
    macd_s = macd.ewm(span=9, adjust=False, min_periods=9).mean()
    macd_h = macd - macd_s
    return df.index.map(macd), df.index.map(macd_s), df.index.map(macd_h)


def preprocess(df):
    df["volume"] = df.volume.astype(float).round(2)
    df["feature_close"] = df.close
    df["feature_open"] = df.open
    df["feature_high"] = df.high
    df["feature_low"] = df.low
    df["feature_volume"] = df.volume
    df["feature_SMA"] = SMA(df, 50)
    df["feature_MiddleBand"], df["feature_LowerBand"] = BBANDS(df, 50)
    df["feature_RSI"] = RSI(df, periods=14)
    df["feature_MACD"], df["feature_MACD_S"], df["feature_MACD_H"] = MACD(df)
    df = df.dropna()

    return df

In [ ]:
# Reward function

def reward_function(history):
    # history["portfolio_valuation", -2] - history["portfolio_valuation", -2]/100
    init_value = history["portfolio_valuation", 0]
    # prev_value = history["portfolio_valuation", -2]
    curr_value = history["portfolio_valuation", -1]

    pc = np.sum(np.diff(history["position"]) != 0)
    if 20 < pc <= 30:
        r = -80
    elif 10 < pc <= 20:
        r = -90
    elif 0 < pc <= 10:
        r = -100
    else:
        r = 100 * (curr_value / init_value - 1)
    
    return r

In [11]:
# Enviornment

env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="D:/Destktop/PyCharm_Projects/sb3/data/train/day/**/**/*.pkl",
    preprocess=preprocess,
    # reward_function=reward_function,
    positions=[-1, 0, 1],
    trading_fees=0.0001,
    borrow_interest_rate=0.0003,
)
env.unwrapped.add_metric(
    "Position Changes", lambda history: np.sum(np.diff(history["position"]) != 0)
)
env.unwrapped.add_metric("Episode Length", lambda history: len(history["position"]))

In [12]:
# Train

warnings.filterwarnings("ignore", category=TqdmExperimentalWarning)
model = QRDQN(
    MlpPolicy,
    env,
    buffer_size=100000000,  # 1000000
    batch_size=32,
    verbose=0,
    device="auto",
    tensorboard_log="./tensorboard/",
)
model.learn(total_timesteps=5000000, progress_bar=True)

vec_env = model.get_env()
obs = vec_env.reset()

Output()

Market Return : -0.49%   |   Portfolio Return : -14.26%   |   Position Changes : 887   |   Episode Length : 1330   
|

Market Return : -7.61%   |   Portfolio Return : -21.60%   |   Position Changes : 902   |   Episode Length : 1330   
|

Market Return : -1.68%   |   Portfolio Return : -13.66%   |   Position Changes : 892   |   Episode Length : 1330   
|

Market Return : -0.55%   |   Portfolio Return : -14.04%   |   Position Changes : 904   |   Episode Length : 1330   
|

...

Market Return :  0.64%   |   Portfolio Return : -5.25%   |   Position Changes : 251   |   Episode Length : 1330   |

Market Return : -1.95%   |   Portfolio Return :  0.49%   |   Position Changes : 303   |   Episode Length : 1330   |

Market Return : -3.02%   |   Portfolio Return : -0.38%   |   Position Changes : 292   |   Episode Length : 1330   |

Market Return : -0.93%   |   Portfolio Return : -10.00%   |   Position Changes : 383   |   Episode Length : 1330   
|

Market Return : -3.14%   |   Portfolio Return : -5.68%   |   Position Changes : 296   |   Episode Length : 1330   |

In [5]:
reward = 0.0
done, truncated = False, False
observation, info = env.reset()

warnings.filterwarnings("ignore", category=UserWarning)
while not done and not truncated:
    action = model.predict(observation, deterministic=True)
    observation, r, done, truncated, info = env.step(action[0])
    reward += r

env.unwrapped.save_for_render(dir="render_logs")

Market Return : 21.51%   |   Portfolio Return : -99.65%   |   Position Changes : 26   |   Episode Length : 44133   |   


In [ ]:
data = pd.read_csv("1.csv", parse_dates=["date"], index_col="date")
env = gym.make(
    "TradingEnv",
    df=data,
    preprocess=preprocess,
    positions=[-1, 0, 1],
    trading_fees=0.0001,
    borrow_interest_rate=0.0003,
)
env.unwrapped.add_metric(
    "Position Changes", lambda history: np.sum(np.diff(history["position"]) != 0)
)
env.unwrapped.add_metric("Episode Length", lambda history: len(history["position"]))

model = QRDQN(
    MlpPolicy,
    env,
    buffer_size=100000000,
    batch_size=32,
    verbose=0,
    device="auto",
)
model.load("./model/QRDQN/1.zip")

vec_env = model.get_env()
obs = vec_env.reset()

reward = 0.0
done = False

warnings.filterwarnings("ignore", category=UserWarning)
while not done:
    action, _state = model.predict(obs, deterministic=True)
    obs, r, done, info = vec_env.step(action)
    reward += r
    vec_env.render("logs")

print(reward)

In [6]:
renderer = Renderer(render_logs_dir="render_logs")
renderer.add_metric(
    "Annual Market Return",
    lambda df: f"{((df['close'].iloc[-1] / df['close'].iloc[0]) ** (pd.Timedelta(days=365) / (df.index.values[-1] - df.index.values[0])) - 1) * 100:0.2f}%",
)
renderer.add_metric(
    "Annual Portfolio Return",
    lambda df: f"{((df['portfolio_valuation'].iloc[-1] / df['portfolio_valuation'].iloc[0]) ** (pd.Timedelta(days=365) / (df.index.values[-1] - df.index.values[0])) - 1) * 100:0.2f}%",
)
renderer.run()

 * Serving Flask app 'gym_trading_env.renderer'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Mar/2024 21:50:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2024 21:50:25] "GET /update_data/2020_02_1m.pkl_2024-03-02_21-28-44.pkl HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2024 21:50:25] "GET /metrics HTTP/1.1" 200 -


In [ ]:
model.save("./model/QRDQN/1.zip")